<a href="https://colab.research.google.com/github/AHamamd150/Zewail-city/blob/main/Custom_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow import keras
from tensorflow.keras import layers 
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential 
from tensorflow.keras.datasets import mnist
import tensorflow as tf

In [2]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train=x_train/255
x_test = x_test/255
x_train=x_train
x_test=x_test


11501568/11490434 [==============================] - 0s 0us/step


# Customize our own Dense Layer

In [3]:

class MyDense(tf.keras.layers.Layer):

  def __init__(self, units):
      super(MyDense, self).__init__()
      self.units = units
      
  def build(self, input_shape):
      self.w = self.add_weight(shape=(input_shape[-1], self.units),
                               initializer='zeros',
                               trainable=True)
      self.b = self.add_weight(shape=(self.units,),
                               initializer='zeros',
                               trainable=True)

  def call(self, inputs):
      y =  tf.matmul(inputs, self.w) + self.b
      return y
      
class MyRule(tf.keras.layers.Layer):
  def __init__(self):
    super(MyRule,self).__init__()

  def call(self,x):
    return tf.math.maximum(-0.1,x)

class Mysoftmax(tf.keras.layers.Layer):
  def __init__(self):
    super(Mysoftmax,self).__init__()

  def call(self,x):
    return tf.math.exp(x)/tf.math.reduce_sum(tf.math.exp(x)) 


# Our custom model 

In [4]:
class Mymodel(keras.Model):
    def __init__(self,outputs=10):
         super(Mymodel,self).__init__()
         self.outputs=outputs
         self.dense1 = tf.keras.layers.Flatten(input_shape=(28,28))
         self.dense2= MyDense(56)
         self.dense3= MyDense(10)
         self.relu = MyRule()
         self.softmax=Mysoftmax()

    def call(self,inputs):
        x = self.dense1(inputs)
        #x = self.relu(x)
        #x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)
        x = self.softmax(x)

        return x

model = Mymodel()

In [ ]:
model.compile(optimizer= keras.optimizers.Adam(),loss=keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=20,batch_size=256,validation_split=0.2)

Epoch 1/20
188/188 [==============================] - 1s 5ms/step - loss: 0.9312 - accuracy: 0.8186 - val_loss: 0.5044 - val_accuracy: 0.8808
Epoch 2/20
188/188 [==============================] - 1s 4ms/step - loss: 0.4648 - accuracy: 0.8836 - val_loss: 0.3867 - val_accuracy: 0.9021
Epoch 3/20
188/188 [==============================] - 1s 4ms/step - loss: 0.3867 - accuracy: 0.8988 - val_loss: 0.3434 - val_accuracy: 0.9102
Epoch 4/20
188/188 [==============================] - 1s 5ms/step - loss: 0.3505 - accuracy: 0.9053 - val_loss: 0.3205 - val_accuracy: 0.9130
Epoch 5/20
188/188 [==============================] - 1s 4ms/step - loss: 0.3287 - accuracy: 0.9100 - val_loss: 0.3067 - val_accuracy: 0.9181
Epoch 6/20
188/188 [==============================] - 1s 4ms/step - loss: 0.3148 - accuracy: 0.9137 - val_loss: 0.2962 - val_accuracy: 0.9199
Epoch 7/20
188/188 [==============================] - 1s 4ms/step - loss: 0.3041 - accuracy: 0.9165 - val_loss: 0.2910 - val_accuracy: 0.9193
Epoch 

In [ ]:
model.evaluate(x_test,y_test,verbose=0)[-1]

0.9271000027656555

In [7]:
n_epochs = 20
batch_size = 256
n_steps = len(x_train) // batch_size
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.mean_squared_error
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]


In [11]:
def random_batch(X, y, batch_size=32):
  idx = np.random.randint(len(X), size=batch_size) 
  return X[idx], y[idx]

for epoch in range(1, n_epochs + 1): 
  print("Epoch {}/{}".format(epoch, n_epochs))
  for step in range(1, n_steps + 1):
    X_batch, y_batch = random_batch(x_train, y_train)
    with tf.GradientTape() as tape:
      y_pred = model(X_batch, training=True)
      main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
      loss = tf.add_n([main_loss] + model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables)) 
    mean_loss(loss)
    for metric in metrics: 
      metric(y_batch, y_pred)
    print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)  
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
      metric.reset_states()

Epoch 1/20


InvalidArgumentError: ignored